In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import register_keras_serializable
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.losses import mean_squared_error
from sklearn.metrics import r2_score
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler
import joblib


@register_keras_serializable()
class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units

    
# Load your saved Keras model
model = tf.keras.models.load_model('model.keras', custom_objects={'RBFLayer': RBFLayer})
my_scaler = joblib.load('scaler.gz')

In [10]:
# column_names = ["7MSHOT", "6MSHOT", "9MSHOT", "WINGSHOT", "PIVOTSHOT", "FASTBREAKSHOT"]
# input_values = np.array([[1, 1, 1, 1, 1, 1]])
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/sens/RWACC.csv'
# Create a DataFrame with the same column names as the scaler was fitted on
input_df = pd.read_csv(url, delimiter=';')

# Transform the input DataFrame using the loaded scaler
scaled_input_values = my_scaler.transform(input_df)

# Make predictions using the model
predictions = model.predict(scaled_input_values)

# Create a DataFrame to store predictions
predictions_df = pd.DataFrame(predictions, columns=['Predicted_Goal'])

# Concatenate input_df and predictions_df
result_df = pd.concat([input_df, predictions_df], axis=1)


# Save the resulting DataFrame as a new CSV file
result_df.to_csv('predictions_with_input.csv', index=False)

3/3 [==============================] - 0s 2ms/step


In [11]:
predictions

array([[3.2324886],
       [3.0633335],
       [3.4129467],
       [3.7959914],
       [4.165526 ],
       [4.4874544],
       [4.7440996],
       [4.93281  ],
       [5.061439 ],
       [5.142994 ],
       [5.19121  ],
       [3.2324886],
       [3.1810632],
       [3.573688 ],
       [3.9967766],
       [4.402752 ],
       [4.735486 ],
       [4.942688 ],
       [4.989298 ],
       [4.8671846],
       [4.597464 ],
       [4.22453  ],
       [3.2324886],
       [3.5301847],
       [3.9097595],
       [4.262969 ],
       [4.539258 ],
       [4.6942883],
       [4.701552 ],
       [4.559781 ],
       [4.293307 ],
       [3.9452362],
       [3.5660653],
       [3.2324886],
       [1.9461919],
       [2.3341446],
       [2.7941227],
       [3.2893567],
       [3.7640386],
       [4.1524506],
       [4.3935018],
       [4.4463882],
       [4.301883 ],
       [3.9849672],
       [3.2324886],
       [3.2470155],
       [3.379919 ],
       [3.529016 ],
       [3.6754253],
       [3.804879 ],
